# Part-of-Speech Tagging - Thiago Pádua - 2020007066
A tarefa de Part-of-Speech Tagging (POS) consiste em rotular palavras de um texto de acordo com a sua classe gramatical, como substantivo, verbo, adjetivo. Esse processo é fundamental para diversas aplicações em Processamento de Linguagem Natural (PLN), pois permite uma compreensão mais detalhada da estrutura sintática e semântica das sentenças.

Por exemplo, considere a seguinte frase:
"A raposa azul dorme tranquilamente."

    "A" pode receber a tag de artigo definido.
    "raposa" pode receber a tag de substantivo.
    "azul" pode receber a tag de adjetivo.
    "dorme" pode receber a tag de verbo.
    "tranquilamente" pode receber a tag de advérbio.

Neste trabalho, o objetivo é explorar a tarefa de POS Tagging para a língua portuguesa, utilizando o corpus Mac-Morpho. Além disso, será implementado um modelo capaz de classificar palavras com precisão, analisando como o contexto influencia a atribuição de classes gramaticais. Por fim, investigaremos os desafios e as limitações do processo, discutindo as classes que apresentam maior e menor precisão ao longo do experimento.

## Carregamento dos Dados

In [125]:
import itertools
import torch

import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score
import seaborn as sns

In [126]:
def load_text_data(file_path):
    """
    :param file_path: path to the .txt file containing the data
    :return: List of sentences, where each sentence is a list of tuples (word, tag)
    """
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:  # Ignore empty lines
                # Generate a list of tuples (word, tag)
                word_tags = [tuple(word.split('_')) for word in line.split()]
                sentences.append(word_tags)
    return sentences

In [127]:
test_file_path = "macmorpho-v3/macmorpho-test.txt"
train_file_path = "macmorpho-v3/macmorpho-train.txt"
dev_file_path = "macmorpho-v3/macmorpho-dev.txt"

test_data = load_text_data(test_file_path)
train_data = load_text_data(train_file_path)
dev_data = load_text_data(dev_file_path)

In [128]:
for i, sentence in enumerate(test_data[:3]):
    print(f"Sentence {i+1}: {sentence}")

Sentence 1: [('Salto', 'N'), ('sete', 'ADJ')]
Sentence 2: [('O', 'ART'), ('grande', 'ADJ'), ('assunto', 'N'), ('da', 'PREP+ART'), ('semana', 'N'), ('em', 'PREP'), ('Nova', 'NPROP'), ('York', 'NPROP'), ('é', 'V'), ('a', 'ART'), ('edição', 'N'), ('da', 'PREP+ART'), ('revista', 'N'), ('"', 'PU'), ('New', 'NPROP'), ('Yorker', 'NPROP'), ('"', 'PU'), ('que', 'PRO-KS'), ('está', 'V'), ('nas', 'PREP+ART'), ('bancas', 'N'), ('.', 'PU')]
Sentence 3: [('Número', 'N'), ('duplo', 'ADJ'), ('especial', 'ADJ'), (',', 'PU'), ('é', 'V'), ('inteirinho', 'ADJ'), ('dedicado', 'PCP'), ('a', 'PREP'), ('ensaios', 'N'), ('sobre', 'PREP'), ('moda', 'N'), ('.', 'PU')]


## Metodologia

In [129]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("lisaterumi/postagger-portuguese")
model = AutoModelForTokenClassification.from_pretrained("lisaterumi/postagger-portuguese")

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Supress output
;

''

In [130]:
teste = "A cidade mais linda de Minas Gerais é Belo Horizonte".split(sep=' ')
inputs = tokenizer(teste, return_tensors="pt", is_split_into_words=True, padding=True, truncation=True)

# Obter os índices das palavras originais correspondentes a cada token
word_ids = inputs.word_ids(batch_index=0)  # Adicione o batch_index para evitar erros de dimensão

# Move the inputs to the GPU
if device.type == "cuda":
    inputs = {key: value.to(device) for key, value in inputs.items()}

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [131]:
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# Mostrar os tokens e as palavras originais
for token, word_id in zip(tokens, word_ids):
    palavra_original = teste[word_id] if word_id is not None else None
    print(f"Token: {token}, Palavra Original: {palavra_original}")


Token: [CLS], Palavra Original: None
Token: A, Palavra Original: A
Token: cidade, Palavra Original: cidade
Token: mais, Palavra Original: mais
Token: lin, Palavra Original: linda
Token: ##da, Palavra Original: linda
Token: de, Palavra Original: de
Token: Minas, Palavra Original: Minas
Token: Gerais, Palavra Original: Gerais
Token: é, Palavra Original: é
Token: Belo, Palavra Original: Belo
Token: Horizonte, Palavra Original: Horizonte
Token: [SEP], Palavra Original: None


In [132]:
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

predictions = torch.argmax(logits, dim=-1)

# Mapear previsões para palavras originais
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

tags = []
current_word = None
for token, word_id, pred in zip(tokens, word_ids, predictions[0].tolist()):
    if word_id is None:  # Ignore special tokens
        continue

    # Associate tag only with the first token of each word. Repeated ids will be ignored
    if word_id != current_word:
        tags.append(model.config.id2label[pred])
        current_word = word_id

print("Tags preditas para palavras:", list(zip(teste, tags)))

Tags preditas para palavras: [('A', 'ART'), ('cidade', 'N'), ('mais', 'ADV'), ('linda', 'ADJ'), ('de', 'PREP'), ('Minas', 'NPROP'), ('Gerais', 'NPROP'), ('é', 'V'), ('Belo', 'NPROP'), ('Horizonte', 'NPROP')]


In [133]:
def POS_tagging(split_document, tokenizer, model):
    """
    :param split_document: List of List of words
    :param tokenizer: Tokenizer object
    :param model: Model object

    :return: List of tuples (word, tag)
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    return_predictions = []

    for sentence in split_document:
        inputs = tokenizer(sentence, return_tensors="pt", is_split_into_words=True, padding=True, truncation=True)

        # Get the indices of the original words corresponding to each token
        word_ids = inputs.word_ids(batch_index=0)  # batch_index = 0 to avoid dimension errors

        # Move the input to the GPU
        if device.type == "cuda":
            inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

        predictions = torch.argmax(logits, dim=-1)

        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

        tags = []
        current_word = None
        for token, word_id, pred in zip(tokens, word_ids, predictions[0].tolist()):
            if word_id is None:  # Ignore special tokens
                continue

            # Associate tag only with the first token of each word. Repeated ids will be ignored
            if word_id != current_word:
                tags.append(model.config.id2label[pred])
                current_word = word_id

        return_predictions.append(list(zip(sentence, tags)))

    return return_predictions

In [134]:
test_sentences = [[t[0] for t in tup] for tup in test_data]

In [135]:
unified_test = list(itertools.chain(*test_data))
unified_test_words, unified_test_golden_tags = zip(*unified_test)

In [136]:
test_POS = POS_tagging(test_sentences, tokenizer, model)

## Análise dos Resultados

In [153]:
def evaluate(y_true, y_pred, labels):
    """
    Avalia o desempenho do modelo de POS Tagging com várias métricas.

    :param y_true: Lista de tags verdadeiras.
    :param y_pred: Lista de tags preditas pelo modelo.
    :param labels: Lista de classes possíveis.
    """
    # Acurácia geral
    acuracy = accuracy_score(y_true, y_pred)
    print(f"Acurácia geral: {acuracy:.4f}\n")

    # Precisão global (média ponderada)
    precisao_global = precision_score(y_true, y_pred, labels=labels, average='weighted', zero_division=0)
    print(f"Precisão global (ponderada): {precisao_global:.4f}\n")

    # Relatório detalhado por classe (inclui precisão, revocação e F1-Score)
    print("Relatório de classificação por classe:")
    print(classification_report(y_true, y_pred, labels=labels, zero_division=0))


In [154]:
observed_labels = model.config.label2id.keys()
observed_labels = list(observed_labels)

unified_test_results = list(itertools.chain(*test_POS))
unified_test_results_tags = [t[1] for t in unified_test_results]

print("Size of unified_test_golden_tags: ", len(unified_test_golden_tags))
print("Size of unified_test_results_tags: ", len(unified_test_results_tags))
print(observed_labels)

Size of unified_test_golden_tags:  178373
Size of unified_test_results_tags:  178373
['<pad>', 'ADJ', 'ADV', 'ADV-KS', 'ART', 'CUR', 'IN', 'KC', 'KS', 'N', 'NPROP', 'NUM', 'PCP', 'PDEN', 'PREP', 'PREP+ADV', 'PREP+ART', 'PREP+PRO-KS', 'PREP+PROADJ', 'PREP+PROPESS', 'PREP+PROSUB', 'PRO-KS', 'PROADJ', 'PROPESS', 'PROSUB', 'PU', 'V']


In [155]:
evaluate(unified_test_golden_tags, unified_test_results_tags, observed_labels)

Acurácia geral: 0.9684

Precisão global (ponderada): 0.9686

Relatório de classificação por classe:
              precision    recall  f1-score   support

       <pad>       0.00      0.00      0.00         0
         ADJ       0.96      0.96      0.96      8554
         ADV       0.94      0.90      0.92      5446
      ADV-KS       0.85      0.85      0.85       230
         ART       0.99      0.99      0.99     12580
         CUR       0.00      0.00      0.00       296
          IN       0.55      0.72      0.62        98
          KC       0.98      0.98      0.98      4531
          KS       0.94      0.92      0.93      2538
           N       0.97      0.94      0.96     36542
       NPROP       0.98      0.97      0.97     15936
         NUM       0.68      0.96      0.79      2541
         PCP       0.97      0.97      0.97      3640
        PDEN       0.82      0.92      0.87      1092
        PREP       0.98      0.99      0.98     16778
    PREP+ADV       0.96      0.87  